In [9]:
import os
import glob
import requests
import pandas as pd
import time
import json
from google.colab import drive
################################################################################
## Opening coordinate CSV file
################################################################################
drive.mount('/Dataset/')
#filename = r"C:\Users\Amal\Desktop\Uber_Data_Set\Weekdays\regions_coordinates.csv"
# Read coordinates CSV without index
coordinate_data = pd.read_csv('/Dataset/MyDrive/Dataset/Uber_Dataset/regions_coordinates.csv').filter(['x','y'], axis=1)
#coordinate_data = pd.read_csv(filename).filter(['x','y'], axis=1)
# New arrays for coordinates for fast access
x_data = coordinate_data['x']
y_data = coordinate_data['y']
################################################################################
################################################################################
## Opening dataset files and read it
################################################################################
path = r"/Dataset/MyDrive/Dataset/Uber_Dataset/Dataset-2019-cairo"
#path = r"C:\Users\Amal\Desktop\Uber_Data_Set\Weekdays\dataset"
# Reading data-set CSV files
all_files = glob.glob(os.path.join(path, "*.csv"))
# Create empty list
li = []
# Reading file by file
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
# Converting to pandas data-frame
data = pd.concat(li, axis=0, ignore_index=True)
# Delete unnecessary columns
del data["geometric_mean_travel_time"]
del data["geometric_standard_deviation_travel_time"]
################################################################################
################################################################################
# Pre Data preparation
################################################################################
# New arrays for sourceid & dstid for fast access
sourceid = data['sourceid']
dstid = data['dstid']
#sourceid.values.tolist()
# New Lists for latitude and altitude for source and distination for fast access
source_lng = []
source_lat = []
dst_lng = []
dst_lat = []
distance = []
# Create table of all available regions in cairo for calculating distance matrix
cairo_coordinates = coordinate_data['x'].astype(str) + ',' + coordinate_data['y'].astype(str) + ';'
# Flatten coordinates to a single string for request
cairo_coordinates = ''.join(cairo_coordinates)
# Remove last semi-colon to avoid errors while requesting data from server
cairo_coordinates = cairo_coordinates[0:len(cairo_coordinates)-2]
# Record start time
T = time.time()
################################################################################
# This code is just for colab run only
################################################################################
dist_mat_file = open('/Dataset/MyDrive/Dataset/Uber_Dataset/dist_mat.json')
dist_mat = json.load(dist_mat_file)
# Local Server OSRM
#url = "http://127.0.0.1:5000/table/v1/driving/"
# Request from OSRM server
#r = requests.get(url+cairo_coordinates+'?annotations=distance')
# Reading distance matrix from JSON output
#dist_mat = r.json()['distances']
# Record time taken by server
#server_time = time.time() - T
#print("The server took: " + str(round(server_time,2)) +  " seconds.")
################################################################################
################################################################################
# Data preparation
################################################################################
# Record start time
T = time.time()
# Add coordinates
for i in range(len(sourceid)):
    s = sourceid[i]-1
    d = dstid[i]-1
    source_lng.append(x_data[s])
    source_lat.append(y_data[s])
    dst_lng.append(x_data[d])
    dst_lat.append(y_data[d])
    distance.append(dist_mat[s][d])
# Record time taken by for data-preparation
datapreparation_time = time.time() - T
print("Data preparation took: " + str(round(datapreparation_time,2)) +  " seconds.")
# Introduce New columns in the dataframe
data['lng_s'] = source_lng
data['lat_s'] = source_lat
data['lng_d'] = dst_lng
data['lat_d'] = dst_lat
data['distance'] = distance
# Delete extra columns
del data["sourceid"]
del data["dstid"]
# Re-organize dataframe columns
data = data.reindex(columns=['lng_s', 'lat_s', 'lng_d', 'lat_d', 'dow', 'distance', 'mean_travel_time', 'standard_deviation_travel_time'])
# Save dataset
#data.to_csv('prepared_data.csv')
data.to_csv('/Dataset/MyDrive/Dataset/Uber_Dataset/prepared_dataset.csv')
!cp prepared_dataset.csv "/Dataset/MyDrive/Dataset/Uber_Dataset/prepared_dataset.csv"
################################################################################
################################################################################
print("Finished")

Drive already mounted at /Dataset/; to attempt to forcibly remount, call drive.mount("/Dataset/", force_remount=True).
Data preparation took: 92.66 seconds.
cp: cannot stat 'prepared_dataset.csv': No such file or directory
Finished
